# WSW Crash Finder

Iterate through folders, looking for possible crashes in all supported GPS file types.

Copyright 2024 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

This is very crude and simply looks for Doppler speeds in excess of 30 m/s (i.e. 108 km/h or 58.32 knots).

Such speeds are currently unrealistic for a windsurfer.

In [1]:
import os
import sys

import numpy as np

from shapely.geometry import Polygon, Point

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [2]:
roadCoordinates = [
    (50.57231527046923, -2.462386728786816),
    (50.57928829629842, -2.4686713543672),
    (50.58166491593653, -2.470184611133845),
    (50.58627092109072, -2.471909956383185),
    (50.58628653573243, -2.473113392996906),
    (50.5815763678751, -2.471438610181945),
    (50.57878807423553, -2.469637719414264),
    (50.57178034261953, -2.463143746776421),
    (50.57054567361534, -2.460492898345708),
    (50.57104838137774, -2.460104740541679),
    (50.57231527046923, -2.462386728786816)
]

road = Polygon(roadCoordinates)

crashThreshold = 20 * 1852 / 3600

def findSpikes():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, '..', 'wsw-data')

    errors = {}
    spikes = {}
    
    totFiles = 0
    totPoints = 0
    totDist = 0

    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            #if ext and ext in ['.sbn', '.sbp', '.ubx']:
            if ext and ext in ['.oao']:
                filePath = os.path.join(root, file)
                reader = getFileReader(filePath)
                try:
                    # Some legacy ESP-GPS files contain bad checksums
                    if ext == '.ubx':
                        reader.load(ignoreChecksums=True)
                    else:
                        reader.load()
                    
                    # Process all tracks within the file
                    for track in reader.tracks:
                        maxSog = 0

                        points = (np.argwhere(track.data['hdop'] > 20))
                        for i in points:
                            lat = track.data['lat'][i[0]]
                            lon = track.data['lon'][i[0]]
                            point = Point(lat, lon)

                            if not road.contains(point) and track.data['ehpe'][i[0]] < 50:
                                sog =  track.data['sog'][i[0]]
                                if sog > maxSog:
                                    maxSog = sog
                    
                        if maxSog > crashThreshold:
                            msg = '{:.02f},{:.02f},{:.02f}'.format(
                                maxSog, maxSog * 3600 / 1852, maxSog * 3600 / 1000)
                            spikes[filePath.replace(projdir + '/', '')] = msg
                            print('H', end='')

                        print('.', end='')
                        
                        totPoints += len(track.data['sog'])
                        if ext in ['.oao']:
                            totDist += track.data['sog'].sum() / 5
                        else:
                            totDist += track.data['sog'].sum()

                    totFiles += 1

                except Exception:
                    errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
                    print('E', end='')

    print(os.linesep * 2 + 'Summary: files = {:,}, points = {:,}, distance = {:,.02f} km'.format(
        totFiles, totPoints, totDist / 1000))

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

    if len(spikes) > 0:
        print(os.linesep * 2 + 'Crashes:')
        for spike in spikes:
            print(spike + ',' + spikes[spike])

SyntaxError: invalid syntax (2283858296.py, line 56)

In [ ]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    findSpikes()
    
    print(os.linesep + 'All done!')